<a href="https://colab.research.google.com/github/SteffiJF/ForecastingRedWineRankings/blob/main/WinePreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Merging the data from sales, ranking, and products, and making two files, one for red wine with less than 75 g sugar and one for redwine with less than 9 g sugar. These two files are concatenated in WineMaster.ipynb. Most of the code in this file was produced for my specialization project and is reused here.

In [2]:
!pip install "dask[dataframe]" 
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
import numpy as np
from datetime import datetime
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from datetime import datetime
from numpy import array
%matplotlib inline

     |████████████████████████████████| 118 kB 4.4 MB/s 


Downloading data from Google Disk and merging the files (you need to upload the files into your own Google Disk and change the path to use)

In [3]:
#Downloading sales data
sales = pd.read_csv('/content/drive/MyDrive/vindata/Salgsstatistikk_for_dbe_import_2019_11_30.csv', sep=';',  parse_dates = [['År', 'Måned']], low_memory=False)
#Downloading ranking data
ranks = pd.read_csv('/content/drive/MyDrive/vindata/Rangeringsrapport_for_dbe_import_2019_11_30.csv', sep=';',  parse_dates = [['År', 'Måned']], low_memory=False)
#Downloading product data
products = pd.read_csv('/content/drive/MyDrive/vindata/Products_for_database_import_2019_11_30.csv',sep=';',error_bad_lines=False)

#Splitting dataset into seperate product types 
redSales=sales[sales['Varetype']=="Rødvin"]
whiteSales=sales[sales['Varetype']=="Hvitvin"]
redRanks = ranks[ranks['Produktgruppe']=='Rødvin < 9 g sukker per liter']

#Changing column names to match data frames
products = products.rename(columns={"VMP ID": "Artikkelnr", "Alkohol %": "Alkoholprosent"})

#Keeping interesting columns
smallRedSales=redSales[['År_Måned','Artikkelnr','Liter denne måned i år', 'Salgspris', 'Land', 'Distrikt', 'Årgang', 'Volum', 'Alkoholprosent', 'Utvalg',]]
smallRanks=redRanks[['År_Måned','Artikkelnr', 'Rangering', 'Status',	'Styringstall', 'Netto Salg', 'Produktgruppe', 'Segmentpris', 'Fredet','Minimum', 'Maksimum']]
smallProducts=products[['Artikkelnr', 'Årgang', 'Alkoholprosent', 'Emballasjetype']]

#Converting to dask dataframe for merging
dfRedSales = dd.from_pandas(smallRedSales, npartitions=10)
dfRanks = dd.from_pandas(smallRanks, npartitions=10)
dfProducts = dd.from_pandas(smallProducts, npartitions=10)

# Merge the csv files. Changing order of ranks and sales compared to last project.
df = dd.merge(dfRanks, dfRedSales, how='left', on=['Artikkelnr', 'År_Måned'])
df = dd.merge(df, dfProducts, how='left', on=['Artikkelnr','Årgang'])

#Converting back to pandas
df=df.compute()

#Sorting data
df=df.sort_values(by=['År_Måned','Land', 'Artikkelnr', 'Årgang', 'Distrikt'])

Adding relevant columns and removing irrelevant rows and NaN values

In [4]:
#Summing up total amount of wine sold per article nr and adding as column
totalLiters=df.groupby(['Artikkelnr'])['Liter denne måned i år'].sum()
totalLiters=totalLiters.to_dict()
df['Artikkelnr']
df['Liter totalt']= df['Artikkelnr'].map(totalLiters)

#Using Alkoholprosent_x if not NaN, otherwise using Alkoholprosent_y
df['Alkoholprosent'] = np.where(df['Alkoholprosent_x'].notnull(), df['Alkoholprosent_x'], df['Alkoholprosent_y'])
df=df.drop(['Alkoholprosent_x','Alkoholprosent_y'], axis='columns')

#Adding a column for amount of alcohol divided by sales price
df['Alk/Pris'] = df['Alkoholprosent']*df['Volum']/df['Salgspris']

#Adding a column for volume divided by sales price
df['Vol/Pris'] = df['Volum']/df['Salgspris']

#Changing name of column to avoid space
df = df.rename(columns={'Liter denne måned i år': 'Liter'})

#Removes true duplicates 
df=df.drop_duplicates()

#Removing infinite values
df=df.replace([np.inf, -np.inf], np.nan)

#Making a copy of the dataframe with multiindex
df2 = df.set_index(['Artikkelnr','År_Måned']).sort_index()

Removing duplicates

In [ ]:
#Handling duplicates

#Finding dates that are duplicated in list of dates and returning
#a string that can be used as index for dataframe
def dupDates(dates):
  seen = {}
  dupes = []

  for i in dates:
      if i not in seen:
          seen[i] = 1
      else:
          if seen[i] == 1:
              dupes.append(i)
          seen[i] += 1
  for i in range(0,len(dupes)):
    dupes[i]=str(dupes[i])
    dupes[i]=dupes[i].replace('-01T00:00:00.000000000','')
  return dupes

#Finds the first duplicate and returns article number and date.
#Helps initiate new dataframe containing only duplicated rows
def arbitrary_duplicate(df2, articles):
  if articles:
    for i in articles:
      dates=df2.loc[i].index.values
      if len(dates)-len(np.unique(dates))!=0:
        dupes=dupDates(dates)
        return i, dupes[0]
  return 0, 0


#Finding articlenrs with duplicates and returning a tuple
#with duplicated article number and date
def duplicate_index(df2,dfDupes, articles):
  dupRows= []
  for i in articles:
    dates=df2.loc[i].index.values
    if len(dates)-len(np.unique(dates))!=0:
      dupes=dupDates(dates)
      for j in range(0,len(dupes)):
        dfDupes=dfDupes.append(df2.loc[i][dupes[j]])
        d=(i,dupes[j])
        dupRows.append(d)
  return dupRows 


#Identifying the columns that vary in the duplicated rows
def nonduplicated_columns(df):
    my_cols = []
    for col in df.columns:
        if df[col].nunique(dropna=False) > 1:
            my_cols.append(list(df.columns).index(col))
    return my_cols

#Printing the columns and amount of times that column is the
#reason for duplicated rows
def duplicates():
  duplicatedColumns=np.zeros(len(df2.columns))

  for art,date in dupRows:
    #print(art, date)
    dupCols=nonduplicated_columns(df2.loc[art][date])
    for i in range(0,len(dupCols)):
      duplicatedColumns[dupCols[i]]+=1

  for i in range(0,len(duplicatedColumns)):
    print(df2.columns[i],duplicatedColumns[i])

#Duplicated rows caused by different amounts of liter sold per month are often 
#caused by multiple distributors, these rows will be handled by adding the Liter 
#columns to one row and removing the rest of the duplicated rows
def remove_duplicates(df, df2, dupRows):
  for art,date in dupRows:
      dupCols=nonduplicated_columns(df2.loc[art][date])
      indices=df.index[(df['Artikkelnr'] == art) & (df['År_Måned'] == date)].tolist()
      if 8 in dupCols:
        print(indices)
        df.loc[indices[0],'Liter']+=df.loc[indices[1]][2]
        df=df.drop(indices[1])
      else:
        for i in range(1,len(indices)):
          df=df.drop(indices[i])
  return df


#List of article numbers
articles=df['Artikkelnr'].tolist()
articles=list(dict.fromkeys(articles))

#Resetting index to remove problem of rows having same index
df = df.reset_index(drop=True)

#Setting up new dataframe for duplicated rows, starting with arbitrary duplicate
dart, ddate = arbitrary_duplicate(df2, articles)

if dart != 0:
  dfDupes=df2.loc[dart][ddate]
  
  #Tuples with duplicate articles and dates  
  dupRows=duplicate_index(df2,dfDupes, articles)
  
  #Removing duplicates
  df= remove_duplicates(df, df2, dupRows)

#Making a copy of the dataframe with multiindex
df2 = df.set_index(['Artikkelnr','År_Måned']).sort_index()

In [6]:
df.to_csv('RedWines9.csv')